In [284]:
import numpy as np
import wandb
import pandas as pd
import datetime

In [285]:
def get_h36m_mpjpe_results(
    artifact_name: str, record: str = "test/test_res", exp: str = "human36m"
):
    api = wandb.Api()
    artifact = api.artifact(artifact_name)

    runs = artifact.used_by()
    runs = filter(
        lambda x: x.state not in ["crashed", "killed", "failed"]
        and x.summary["_timestamp"] > datetime.datetime(2022, 10, 17).timestamp()
        and exp in x.name,
        runs,
    )

    mpjpes = np.array([run.summary[record] for run in runs])

    return mpjpes


def describe(data, title="cGNF", sigfig: dict = dict(mean=1, std=2)):
    print(title)
    df = pd.DataFrame(data)
    return (
        df.explode(list(df.columns))
        .astype("float")
        .describe()
        .loc[["mean", "std"]]
        .T.round(sigfig)
        .T
    )

In [286]:
def get_h36m_calibration_results(
    artifact_name: str, record: str = "test/test_res", exp: str = "calibration", idx=0
):
    api = wandb.Api()
    artifact = api.artifact(artifact_name)

    runs = artifact.used_by()
    runs = filter(
        lambda x: x.state not in ["crashed", "killed", "failed"]
        and x.summary["_timestamp"] > datetime.datetime(2022, 10, 17).timestamp()
        and exp in x.name,
        runs,
    )

    mpjpes = np.array([run.history()[record][idx] for run in runs])

    return mpjpes

In [287]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best"
)
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best", "occl/best_mpjpe"
)

In [288]:
describe(mpjpes)

cGNF


,test,hard,occl
mean,57.50,87.30,47.00
std,0.06,0.13,0.18


In [289]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results("ppierzc/propose_human36m/mpii-prod:best")
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod:best", "occl/best_mpjpe"
)

In [290]:
describe(mpjpes, title="cGNF w Lsample")

cGNF w Lsample


,test,hard,occl
mean,53.00,79.30,41.80
std,0.06,0.05,0.04


In [291]:
mpjpes = {}

mpjpes["test"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best"
)
mpjpes["hard"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best", "hard/test_res"
)
mpjpes["occl"] = get_h36m_mpjpe_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best", "occl/best_mpjpe"
)

In [292]:
describe(mpjpes, title="cGNF xlarge w Lsample")

cGNF xlarge w Lsample


,test,hard,occl
mean,48.50,72.60,39.90
std,0.02,0.09,0.05


In [293]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-xlarge:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [294]:
describe(cal, title="cGNF xlarge w Lsample", sigfig=dict(mean=2, std=10))

cGNF xlarge w Lsample


,test,occl
mean,0.230000,0.070000
std,0.000305,0.000539


In [295]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [296]:
describe(cal, title="cGNF w Lsample", sigfig=dict(mean=2, std=10))

cGNF w Lsample


,test,occl
mean,0.080000,0.030000
std,0.000046,0.000224


In [298]:
cal = {}

cal["test"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best",
    record="calibration_score",
    exp="calibration",
)
cal["occl"] = get_h36m_calibration_results(
    "ppierzc/propose_human36m/mpii-prod-no-mode:best",
    record="calibration_score",
    exp="calibration",
    idx=1,
)

In [299]:
describe(cal, title="cGNF", sigfig=dict(mean=2, std=10))

cGNF


,test,occl
mean,0.080000,0.070000
std,0.000244,0.000506


In [300]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={"display_name": {"$regex": "^wehrbein_calibration_1.*"}},
    )
)
eces = np.array([run.summary["calibration_score"] for run in runs])

cal = {}
cal["test"] = eces

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={"display_name": {"$regex": "^wehrbein_calibration_occl_1.*"}},
    )
)
eces = np.array([run.summary["calibration_score"] for run in runs])
cal["occl"] = eces

In [301]:
describe(cal, title="Wehrbein", sigfig=dict(mean=2, std=10))

Wehrbein


,test,occl
mean,0.180000,0.260000
std,0.000552,0.000441


In [302]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^gaussian_calibration_1.*"},
            "state": "finished",
        },
    )
)
eces = np.array([run.summary["ECE"] for run in runs])

cal = {}
cal["NLL"] = eces

In [303]:
describe(cal, title="Gaussian", sigfig=dict(mean=2, std=10))

Gaussian


,NLL
mean,0.07000
std,0.00001


In [304]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^sharma_calibration_1.*"},
            "state": "finished",
        },
    )
)
eces = np.array([run.summary["calibration_score"] for run in runs])

cal = {}
cal["NLL"] = eces

In [305]:
describe(cal, title="Sharma", sigfig=dict(mean=2, std=10))

Sharma


,NLL
mean,0.360000
std,0.000286


In [306]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^wehrbein_occl_.*"},
            "state": "finished",
        },
    )
)
mpjpe = np.array([run.summary["occlusion"] for run in runs])

mpjpes = {}
mpjpes["Occl"] = mpjpe

In [307]:
describe(mpjpes, title="Wehrbein MPJPE occl", sigfig=dict(mean=1, std=2))

Wehrbein MPJPE occl


,Occl
mean,51.10
std,0.13


In [308]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^gaussian_mpjpe_calibration_.*"},
            "state": "finished",
        },
    )
)
ece = np.array([run.summary["ECE"] for run in runs])

eces = {}
eces["mpjpe"] = ece

In [309]:
describe(eces, title="Gaussian MPJPE", sigfig=dict(mean=2, std=10))

Gaussian MPJPE


,mpjpe
mean,0.420000
std,0.000004


In [310]:
api = wandb.Api()

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^gaussian_mpjpe_nll.*"},
            "state": "finished",
        },
    )
)
mpjpe = np.array([run.summary["minMPJPE"] for run in runs])

mpjpes = {}
mpjpes["NLL"] = mpjpe

runs = list(
    api.runs(
        path="ppierzc/propose_human36m",
        filters={
            "display_name": {"$regex": "^gaussian_mpjpe_mpjpe.*"},
            "state": "finished",
        },
    )
)
mpjpe = np.array([run.summary["minMPJPE"] for run in runs])
mpjpes["minMPJPE"] = mpjpe

In [314]:
describe(mpjpes, title='Gaussian', sigfig=dict(NLL=10, minMPJPE=10))

Gaussian


,NLL,minMPJPE
mean,60.139289,54.775911
std,0.001726,0.001843


In [312]:
mpjpes

{'NLL': array([60.13749226, 60.14093531, 60.13943881]),
 'minMPJPE': array([54.77691421, 54.77378377, 54.77703458])}